In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_sm

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

**Pipelines**

In [4]:
print('Pipelines:')
print(nlp.pipe_names)

Pipelines:
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [11]:
df = pd.read_csv('/content/dataset_IMDB.csv',delimiter=',', nrows = 10000)

In [12]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
df['sentiment'] = pd.Series(df['sentiment']=='positive')
df['sentiment'] = pd.Series([int(x) for x in df['sentiment']])
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [14]:
df['sentiment'].value_counts()

1    5028
0    4972
Name: sentiment, dtype: int64

**Number of positive and negative reviews are almost same**

### **Creating doc object**

In [15]:
def create_doc(text):
  doc = nlp(text)
  return doc

In [16]:
doc = create_doc(df['review'][0])

for token in doc:
  print(token.text,token.lemma_,token.pos_,token.tag_,token.dep_,token.shape_,token.is_alpha,token.is_stop)
  break

One one NUM CD nsubj Xxx True True


**spacy.explain() to understand the meaning of tags**

In [17]:
spacy.explain('SCONJ')

'subordinating conjunction'

### **Lemmatization**

In [18]:
def lemmatizer(doc):
  tokens = []
  for token in doc:
    if token.lemma_ != '-PRON-':
      t = token.lemma_.lower().strip()
    else:
      t = token.lower()
    tokens.append(t)
  return tokens

In [19]:
for token in doc[:10]:
  print(token.text,token.lemma_)

One one
of of
the the
other other
reviewers reviewer
has have
mentioned mention
that that
after after
watching watch


> The provided trained pipelines already include all the required tables, but if we are creating new pipelines, we’ll probably want to install spacy-lookups-data to provide the data when the lemmatizer is initialized.

In [20]:
for sent in doc.sents:
  print(sent)

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked.
They are right, as this is exactly what happened with me.<br /><br />The
first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO.
Trust me, this is not a show for the faint hearted or timid.
This show pulls no punches with regards to drugs, sex or violence.
Its is hardcore, in the classic use of the word.<br /><br
/>It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary.
It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda.
Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....
so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

### **Data preprocessing and visualization**

In [21]:
from spacy.lang.en.stop_words import STOP_WORDS

In [22]:
stopwords = list(STOP_WORDS)
print('No of stopwords:', len(stopwords))

No of stopwords: 326


In [23]:
from spacy import displacy 

displacy.render(doc,style='dep',jupyter=True,options={'distance': 40})

In [24]:
displacy.render(doc,style='ent',jupyter=True,options={'distance':40})

### **Remove stopwords and punctuations**

In [25]:
import string

def text_cleaner(tokens):
  punctuations = string.punctuation
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punctuations:
      cleaned_tokens.append(token)

  return cleaned_tokens

### **Text data cleaning (lemmatization + removal of stopwords and punctuations)**

In [26]:
def text_cleaning(text):
  doc = nlp(text)

  # Lemmatization
  lemmatized_tokens = lemmatizer(doc)

  # Remove stop words and punctuations
  cleaned_data = text_cleaner(lemmatized_tokens)

  return cleaned_data

### **Classifier and vectorizer**

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

classifier = LinearSVC()
vectorizer = TfidfVectorizer(tokenizer = text_cleaning)

### **Train test split**

In [28]:
from sklearn.model_selection import train_test_split

data = df['review']
label = df['sentiment']

X_train,X_test,Y_train,Y_test = train_test_split(data,label,test_size=0.2)

In [29]:
X_train.shape,X_test.shape

((8000,), (2000,))

In [30]:
from sklearn.pipeline import Pipeline

clf = Pipeline([('Vectorizer',vectorizer),('Classifier',classifier)])

In [31]:
clf.fit(X_train,Y_train)

Pipeline(steps=[('Vectorizer',
                 TfidfVectorizer(tokenizer=<function text_cleaning at 0x7fcf2c5068c0>)),
                ('Classifier', LinearSVC())])

In [32]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

Y_pred = clf.predict(X_test)

print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.90      0.85      0.88      1001
           1       0.86      0.90      0.88       999

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



In [33]:
print(confusion_matrix(Y_test,Y_pred))

[[855 146]
 [ 96 903]]


#### **Testing the model on data**

In [34]:
clf.predict(['worth watching','Good movie','Total waste of time','Poor cinematography'])

array([1, 1, 0, 0])